In [18]:
from transformers import pipeline, set_seed
from torch import tensor

generator = pipeline('text-generation', model='gpt2')
set_seed(42)

generator("Hello, I'm a language model", max_length=30, num_return_sequences=5)

Using pad_token, but it is not set yet.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Hello, I'm a language modeler. I want it to help you to read and understand better the language models you're using, how they work"},
 {'generated_text': "Hello, I'm a language modeler in Google and I'm doing a lot of research on language languages. Sometimes, it is kind of awkward to"},
 {'generated_text': "Hello, I'm a language model scientist! My dream for this project is for students to be able to play with our favorite words and sentences from these"},
 {'generated_text': 'Hello, I\'m a language modeler."\n\nKhan\'s answer is that "There are probably some more fundamental questions to be answered before it'},
 {'generated_text': "Hello, I'm a language model teacher, and my focus is on making a beautiful language (and, yes, learning a language is great, if"}]

In [562]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

'sinan' in tokenizer.get_vocab()

False

In [563]:
tokenizer.convert_ids_to_tokens(tokenizer.encode('Sinan loves a beautiful day'))

['Sin', 'an', 'Ġloves', 'Ġa', 'Ġbeautiful', 'Ġday']

In [564]:
tokenizer.encode('Sinan loves a beautiful day')

[46200, 272, 10408, 257, 4950, 1110]

In [565]:
encoded = tokenizer.encode('Sinan loves a beautiful day', return_tensors='pt')

encoded

tensor([[46200,   272, 10408,   257,  4950,  1110]])

In [566]:
model.transformer.wte(encoded).shape

torch.Size([1, 6, 1024])

In [ ]:
model.transformer.wpe(tensor([0, 1, 2, 3, 4, 5]).reshape(1, 6)).shape

In [ ]:
initial_input = model.transformer.wte(encoded) + model.transformer.wpe(tensor([0, 1, 2, 3, 4, 5]).reshape(1, 6))

initial_input.shape

In [ ]:
initial_input = model.transformer.drop(initial_input)
initial_input

In [ ]:
model.lm_head

In [ ]:
for module in model.transformer.h:
    initial_input = module(initial_input)[0]
    
initial_input = model.transformer.ln_f(initial_input)

In [ ]:
(initial_input == model(encoded, output_hidden_states=True).hidden_states[-1]).all()

In [ ]:
(model.lm_head(initial_input) == model(encoded).logits).all()  # lm_head

In [20]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [125]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [22]:
num_tokens = tokenizer.encode(to_summarize, return_tensors='pt').shape[1]

num_tokens

412

In [25]:
for sentence in generator(to_summarize +"\nTL;DR:", max_length=num_tokens + 30, num_return_sequences=2):
    print("Original Text: " + sentence['generated_text'] + "\n\n")

Using pad_token, but it is not set yet.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Original Text: When Lizzie Rothwell, an architect in Philadelphia, sent her son to third grade this fall, she stocked his blue L.L. Bean backpack with pencils, wide-ruled paper — and a portable carbon dioxide monitor.

The device gave her a quick way to assess how much fresh air was flowing through the school. Low levels of CO2 would indicate that it was well-ventilated, reducing her son’s odds of catching the coronavirus.

But she quickly discovered that during lunch, CO2 levels in the cafeteria rose to nearly double those recommended by Centers for Disease Control and Prevention. She shared what she’d learned with the principal and asked if students could eat outside instead.

“He expressed surprise that I had any data at all,” she said.

Ms. Rothwell is one of a growing number of parents who are sneaking CO2 monitors into schools in a clandestine effort to make sure their children’s classrooms are safe. Aranet, which makes a monitor popular with parents, says orders have doubled sin

In [27]:
encoded_input = tokenizer('Original Text: ' + to_summarize + '\nTL;DR:', return_tensors='pt')
    
list(map(tokenizer.decode, model.generate(
        input_ids=encoded_input['input_ids'],
        max_length=min(1000, len(encoded_input['input_ids'][0]) + 100),
        temperature=0.7,
        top_k=2,
        no_repeat_ngram_size=3
    )))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Original Text: When Lizzie Rothwell, an architect in Philadelphia, sent her son to third grade this fall, she stocked his blue L.L. Bean backpack with pencils, wide-ruled paper — and a portable carbon dioxide monitor.\n\nThe device gave her a quick way to assess how much fresh air was flowing through the school. Low levels of CO2 would indicate that it was well-ventilated, reducing her son’s odds of catching the coronavirus.\n\nBut she quickly discovered that during lunch, CO2 levels in the cafeteria rose to nearly double those recommended by Centers for Disease Control and Prevention. She shared what she’d learned with the principal and asked if students could eat outside instead.\n\n“He expressed surprise that I had any data at all,” she said.\n\nMs. Rothwell is one of a growing number of parents who are sneaking CO2 monitors into schools in a clandestine effort to make sure their children’s classrooms are safe. Aranet, which makes a monitor popular with parents, says orders have d

In [75]:
import requests

NYTIMES_KEY = 'uy1HSemASbwBaaSskpdSVckRgSLOVKE6'

article =  requests.get(f'https://api.nytimes.com/svc/mostpopular/v2/viewed/1.json?api-key={NYTIMES_KEY}').json()['results'][1]

article


{'uri': 'nyt://article/e1eab467-0f63-566f-9eed-7cea6be9a278',
 'url': 'https://www.nytimes.com/2021/10/09/arts/television/shannen-doherty-breast-cancer-instagram.html',
 'id': 100000008015259,
 'asset_id': 100000008015259,
 'source': 'New York Times',
 'published_date': '2021-10-09',
 'updated': '2021-10-09 15:43:49',
 'section': 'Arts',
 'subsection': 'Television',
 'nytdsection': 'arts',
 'adx_keywords': 'Actors and Actresses;Breast Cancer;Chemotherapy;Television;Doherty, Shannen;Instagram Inc',
 'column': None,
 'byline': 'By Maria Cramer',
 'type': 'Article',
 'title': 'In Candid Photos, Shannen Doherty Shows the Ravages of Breast Cancer',
 'abstract': 'The actress, 50, who has Stage 4 cancer, said she posted the photos to help raise awareness about breast cancer prevention.',
 'des_facet': ['Actors and Actresses',
  'Breast Cancer',
  'Chemotherapy',
  'Television'],
 'org_facet': ['Instagram Inc'],
 'per_facet': ['Doherty, Shannen'],
 'geo_facet': [],
 'media': [{'type': 'image',

In [76]:
print(article['url'])

https://www.nytimes.com/2021/10/09/arts/television/shannen-doherty-breast-cancer-instagram.html


In [77]:
from bs4 import BeautifulSoup

article_body = BeautifulSoup(requests.get(article['url']).text).find('section', {'name': 'articleBody'}).text

print(article_body)

One picture shows the actress Shannen Doherty completely bald, a bloody cotton ball in her nose as she stares straight at the camera, looking almost confrontational.Another is more playful — Ms. Doherty, 50, is in bed wearing Cookie Monster pajamas and a Cookie Monster eye mask. She confesses to how exhausted she is, how the chemotherapy she has had to undergo for Stage 4 breast cancer has left her plagued by bloody noses.“Is it all pretty? NO but it’s truthful and my hope in sharing is that we all become more educated, more familiar with what cancer looks like,” Ms. Doherty wrote on Instagram this week.The images are unsettling to any member of Generation X who remembers her as Brenda Walsh, the feisty, polarizing teenager she played for four years on the hit 1990s show “Beverly Hills, 90210,” which brought her international fame and infamy.Ms. Doherty said she was posting the images as part of Breast Cancer Awareness Month in the hopes that they will jar people into getting mammogram

In [78]:
encoded_input = tokenizer('Original Text: ' + article_body + '\nTL;DR:', return_tensors='pt')['input_ids'][0, -990:].unsqueeze(0)
    
print(list(map(tokenizer.decode, model.generate(
        input_ids=encoded_input,
        temperature=0.7,
        top_k=2,
        no_repeat_ngram_size=3,
        max_length=1024
    )))[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 when she is communicating with you, she is coming from an authentic place.”Ms. Doherty said she learned she had breast cancer in 2015. Since then, she said she has had a mastectomy, as well as radiation and chemotherapy treatment.The photos, which have been viewed about 280,000 times, have elicited comments of sympathy, admiration and praise on her Instagram account, which has more than 1.8 million followers.“Love you Shan,” wrote Ian Ziering, one of her former co-stars on “Beverly Hills, 90210.”“You are a force, Sister!” wrote Kelly Hu, an actress.Ms. Doherty did not often get such adulation when she was younger.In the early 1990s, Ms. Doherty, who was only 19 when she started acting on “90210,” was eviscerated by the press and many in the public who criticized her for smoking in clubs, her tumultuous love life and reports that she was difficult on set.Her character was an outspoken, headstrong and temperamental teenager who had sex with her boyfriend, fought with her friends and reb

In [9]:
generator("""Good morning. = Buenos días.

How old are you? = """, max_length=200)

Using pad_token, but it is not set yet.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Good morning. = Buenos días.\n\nHow old are you? = ????..\n\nWhat was your last name? = ????..\n\nWhere did you live? = Los Angeles, CA =????.......????\n\nHow do you describe the person you are with? = Very good friends\n\nHow tall do you think she is? = 5'5 = ??????.........??????!!\n\nSo what do women REALLY think? = she is beautiful = ????.\n\nAny additional comments? = ????...??\n\nHow do you guys think girls like you? = ????..\n\nHow long does your relationship last? = ????..??\n\nYou guys seem like you are really nice guys for sure. Tell me what I could do to help you!\n\nThank you for the interview. I have been trying my best to connect with you but we are two adults in our 30s here and we can make it"}]

{'input_ids': tensor([[3666, 3290,  468,  257, 1256,  286, 2568,  290,  673,  318,  523]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [163]:
phrase = 'My friend was right about this class. It is so fun!'
encoded_phrase = tokenizer(phrase, return_tensors='pt')

response = model(**encoded_phrase, output_attentions=True, output_hidden_states=True)

len(response.attentions)

12

In [164]:
response.attentions[-1].shape

torch.Size([1, 12, 13, 13])

In [165]:
encoded_phrase['input_ids'].shape

torch.Size([1, 13])

In [185]:
tokens = tokenizer.convert_ids_to_tokens(encoded_phrase['input_ids'][0])

In [198]:
import torch
import pandas as pd

# Layer 9, head 0
arr = response.attentions[9][0][0]

n_digits = 3

attention_df = pd.DataFrame((torch.round(arr * 10**n_digits) / (10**n_digits)).detach()).applymap(float)

attention_df.columns = tokens
attention_df.index = tokens

attention_df


,My,Ġfriend,Ġwas,Ġright,Ġabout,Ġthis,Ġclass,.,ĠIt,Ġis,Ġso,Ġfun,!
My,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġfriend,0.968,0.032,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġwas,0.824,0.145,0.031,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġright,0.979,0.008,0.007,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġabout,0.979,0.008,0.004,0.005,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġthis,0.924,0.031,0.007,0.006,0.016,0.016,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġclass,0.946,0.005,0.001,0.001,0.001,0.002,0.044,0.000,0.000,0.000,0.000,0.000,0.000
.,0.691,0.013,0.003,0.003,0.002,0.006,0.269,0.013,0.000,0.000,0.000,0.000,0.000
ĠIt,0.318,0.003,0.003,0.003,0.006,0.018,0.599,0.018,0.032,0.000,0.000,0.000,0.000
Ġis,0.331,0.006,0.002,0.002,0.003,0.018,0.533,0.013,0.062,0.030,0.000,0.000,0.000


In [167]:
from bertviz import model_view

tokens = tokenizer.convert_ids_to_tokens(encoded_phrase['input_ids'][0]) 
model_view(response.attentions, tokens)

<IPython.core.display.Javascript object>

In [334]:
len(response.hidden_states)

13

In [335]:
response.hidden_states[-1].shape

torch.Size([1, 13, 768])

In [336]:
response.logits.shape

torch.Size([1, 13, 50257])

In [357]:
pd.DataFrame(
    zip(tokens, tokenizer.convert_ids_to_tokens(response.logits.argmax(2)[0])), 
    columns=['Sequence up until', 'Next token with highest probability']
)

,Sequence up until,Next token with highest probability
0,My,Ċ
1,Ġfriend,","
2,Ġwas,Ġa
3,Ġright,.
4,Ġabout,Ġthat
5,Ġthis,.
6,Ġclass,.
7,.,ĠI
8,ĠIt,'s
9,Ġis,Ġa


In [358]:
generator(phrase, max_length=20, num_return_sequences=1, do_sample=False)  # greedy search

Using pad_token, but it is not set yet.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'My friend was right about this class. It is so fun! I love it! I love the'}]

In [ ]:
generator(phrase, max_length=20, num_return_sequences=1, do_sample=True)  # greedy search

In [228]:
## Zero Shot Learning

In [298]:
to_summarize = """This training will focus on how the GPT family of models are used for NLP tasks including abstractive text summarization and natural language generation. The training will begin with an introduction to necessary concepts including masked self attention, language models, and transformers and then build on those concepts to introduce the GPT architecture. We will then move into how GPT is used for multiple natural language processing tasks with hands-on examples of using pre-trained GPT-2 models as well as fine-tuning these models on custom corpora.

GPT models are some of the most relevant NLP architectures today and it is closely related to other important NLP deep learning models like BERT. Both of these models are derived from the newly invented transformer architecture and represent an inflection point in how machines process language and context.

The Natural Language Processing with Next-Generation Transformer Architectures series of online trainings provides a comprehensive overview of state-of-the-art natural language processing (NLP) models including GPT and BERT which are derived from the modern attention-driven transformer architecture and the applications these models are used to solve today. All of the trainings in the series blend theory and application through the combination of visual mathematical explanations, straightforward applicable Python examples within hands-on Jupyter notebook demos, and comprehensive case studies featuring modern problems solvable by NLP models. (Note that at any given time, only a subset of these classes will be scheduled and open for registration.)"""

In [315]:
print(generator(
    f"""Summarization Task:\n{to_summarize}\nTL;DR:""", 
    max_length=512, top_k=3, beams=3, temperature=0.5, no_repeat_ngram_size=3
)[0]['generated_text'])


Using pad_token, but it is not set yet.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Summarization Task:
This training will focus on how the GPT family of models are used for NLP tasks including abstractive text summarization and natural language generation. The training will begin with an introduction to necessary concepts including masked self attention, language models, and transformers and then build on those concepts to introduce the GPT architecture. We will then move into how GPT is used for multiple natural language processing tasks with hands-on examples of using pre-trained GPT-2 models as well as fine-tuning these models on custom corpora.

GPT models are some of the most relevant NLP architectures today and it is closely related to other important NLP deep learning models like BERT. Both of these models are derived from the newly invented transformer architecture and represent an inflection point in how machines process language and context.

The Natural Language Processing with Next-Generation Transformer Architectures series of online trainings provides a

## Few Shot Learning

In [561]:
print(generator("""Sentiment Analysis
Text: I hate it when my phone battery dies.
Sentiment: Negative
###
Text: My day has been really great!
Sentiment: Positive
###
Text: This new music video was so good
Sentiment:""", top_k=2, temperature=0.1, max_length=55)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sentiment Analysis
Text: I hate it when my phone battery dies.
Sentiment: Negative
###
Text: My day has been really great!
Sentiment: Positive
###
Text: This new music video was so good
Sentiment: Positive



In [330]:
print(generator("""Question/Answering
C: Google was founded in 1998 by Larry Page and Sergey Brin while they were Ph.D. students at Stanford University in California. Together they own about 14 percent of its shares and control 56 percent of the stockholder voting power through supervoting stock.
Q: When was Google founded?
A: 1998
###
C: Hugging Face is a company which develops social AI-run chatbot applications. It was established in 2016 by Clement Delangue and Julien Chaumond. The company is based in Brooklyn, New York, United States.
Q: What does Hugging Face develop?
A: social AI-run chatbot applications
###
C: The New York Jets are a professional American football team based in the New York metropolitan area. The Jets compete in the National Football League (NFL) as a member club of the league's American Football Conference (AFC) East division.
Q: In which division are the Jets playing?
A:""", top_k=3, beams=2, max_length=215, temperature=0.3)[0]['generated_text'])

Using pad_token, but it is not set yet.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question/Answering
C: Google was founded in 1998 by Larry Page and Sergey Brin while they were Ph.D. students at Stanford University in California. Together they own about 14 percent of its shares and control 56 percent of the stockholder voting power through supervoting stock.
Q: When was Google founded?
A: 1998
###
C: Hugging Face is a company which develops social AI-run chatbot applications. It was established in 2016 by Clement Delangue and Julien Chaumond. The company is based in Brooklyn, New York, United States.
Q: What does Hugging Face develop?
A: social AI-run chatbot applications
###
C: The New York Jets are a professional American football team based in the New York metropolitan area. The Jets compete in the National Football League (NFL) as a member club of the league's American Football Conference (AFC) East division.
Q: In which division are the Jets playing?
A: In the AFC East.
###



In [332]:
# Same question as before, with no previous examples. Bad response
print(generator("In which division are the Jets playing?", top_k=3, beams=2, max_length=215, temperature=0.3)[0]['generated_text'])

Using pad_token, but it is not set yet.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In which division are the Jets playing?

The Jets are playing in a divisional round of 32.

The Jets are playing in a divisional round of 32.

The Jets are playing in a divisional round of 32.

The Jets are playing in a divisional round of 32.

The Jets are playing in a divisional round of 32.

The Jets are playing in a divisional round of 32.

The Jets are playing in a divisional round of 32.

The Jets are playing in a divisional round of 32.

The Jets are playing in a divisional round of 32.

The Jets are playing in a divisional round of 32.

The Jets are playing in a divisional round of 32.

The Jets are playing in a divisional round of 32.

The Jets are playing in a divisional round of 32.

The Jets are playing in a divisional round of 32.

The Jets are playing in a divisional round


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [34]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 73
  Batch size = 64


{'eval_loss': 4.955022811889648,
 'eval_runtime': 45.7959,
 'eval_samples_per_second': 1.594,
 'eval_steps_per_second': 0.044}

In [35]:
trainer.train()

***** Running training *****
  Num examples = 73
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 9


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=9, training_loss=5.281565772162543, metrics={'train_runtime': 451.8261, 'train_samples_per_second': 0.485, 'train_steps_per_second': 0.02, 'total_flos': 14305738752000.0, 'train_loss': 5.281565772162543, 'epoch': 3.0})

In [36]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 73
  Batch size = 64


{'eval_loss': 4.942089557647705,
 'eval_runtime': 44.55,
 'eval_samples_per_second': 1.639,
 'eval_steps_per_second': 0.045,
 'epoch': 3.0}

In [553]:
from transformers import GPT2Tokenizer, GPT2Model
tokenizer = GPT2Tokenizer.from_pretrained(
    'gpt2',
)
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')

tokenizer.pad_token = tokenizer.eos_token


In [554]:
total_params = 0
for param in model.parameters():
    total_params += torch.numel(param)
    
print(f'Number of params: {total_params:,}')

Number of params: 354,823,168


In [375]:
encoded_input = tokenizer('I am happy', return_tensors='pt')
    
list(map(tokenizer.decode, model.generate(
        input_ids=encoded_input['input_ids'],
        max_length=min(1000, len(encoded_input['input_ids'][0]) + 5),
        temperature=0.7,
        top_k=2,
    repitition_penalty=0.1,
        do_sample=True,
    )))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['I am happy to announce that we have']

## Bias

In [367]:
from transformers import pipeline, set_seed
from torch import tensor

generator = pipeline('text-generation', model='gpt2', tokenizer=tokenizer)
set_seed(0)

In [527]:
generator("The white man works during the day as a", max_length=12, num_return_sequences=5, temperature=0.9, beams=2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The white man works during the day as a police officer.'},
 {'generated_text': 'The white man works during the day as a barber,'},
 {'generated_text': 'The white man works during the day as a waiter. Some'},
 {'generated_text': 'The white man works during the day as a security guard at'},
 {'generated_text': 'The white man works during the day as a waiter, but'}]

In [524]:
generator("The black man works during the day as a", max_length=12, num_return_sequences=5, temperature=0.9, beams=2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The black man works during the day as a "black guy'},
 {'generated_text': 'The black man works during the day as a security guard for'},
 {'generated_text': 'The black man works during the day as a janitor,'},
 {'generated_text': 'The black man works during the day as a security guard in'},
 {'generated_text': 'The black man works during the day as a mechanic. "'}]

In [519]:
generator("The black woman works during the day as a", max_length=12, num_return_sequences=5, temperature=0.9, beams=2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The black woman works during the day as a taxi driver or'},
 {'generated_text': 'The black woman works during the day as a maid until 11'},
 {'generated_text': 'The black woman works during the day as a prostitute and the'},
 {'generated_text': 'The black woman works during the day as a waitress at a'},
 {'generated_text': 'The black woman works during the day as a waitress. "'}]